## 本地数据库方式

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_milvus import Milvus

# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)
splits = text_splitter.split_documents(docs)
print(len(splits))

# 文本嵌入
embed = ZhipuAIEmbeddings(model="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 向量库创建 - 添加 index_params to specify supported index type
vectorstore = Milvus.from_documents(
    documents=splits,  # You should use the splits, not the original docs
    embedding=embed,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    index_params={
        "metric_type": "L2",  # or "IP" depending on your needs
        "index_type": "IVF_FLAT",  # Use a supported index type
        "params": {"nlist": 128}  # IVF_FLAT parameter
    },
    drop_old=True,
)


# 检索
question = "图像识别"
docs = vectorstore.similarity_search(question,k=3)
print(len(docs))
print(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


18
3
10. 注意力机制
10.1. 注意力提示
10.2. 注意力汇聚：Nadaraya-Watson 核回归
10.3. 注意力评分函数
10.4. Bahdanau 注意力
10.5. 多头注意力
10.6. 自注意力和位置编码
10.7. Transformer


11. 优化算法
11.1. 优化和深度学习
11.2. 凸性
11.3. 梯度下降
11.4. 随机梯度下降
11.5. 小批量随机梯度下降
11.6. 动量法
11.7. AdaGrad算法
11.8. RMSProp算法
11.9. Adadelta
11.10. Adam算法
11.11. 学习率调度器


12. 计算性能
12.1. 编译器和解释器
12.2. 异步计算
12.3. 自动并行
12.4. 硬件
12.5. 多GPU训练
12.6. 多GPU的简洁实现
12.7. 参数服务器


13. 计算机视觉
13.1. 图像增广
13.2. 微调
13.3. 目标检测和边界框
13.4. 锚框
13.5. 多尺度目标检测
13.6. 目标检测数据集
13.7. 单发多框检测（SSD）
13.8. 区域卷积神经网络（R-CNN）系列
13.9. 语义分割和数据集
13.10. 转置卷积
13.11. 全卷积网络
13.12. 风格迁移
13.13. 实战 Kaggle 比赛：图像分类 (CIFAR-10)
13.14. 实战Kaggle比赛：狗的品种识别（ImageNet Dogs）


14. 自然语言处理：预训练
14.1. 词嵌入（word2vec）
14.2. 近似训练
14.3. 用于预训练词嵌入的数据集
14.4. 预训练word2vec
14.5. 全局向量的词嵌入（GloVe）
14.6. 子词嵌入
14.7. 词的相似性和类比任务
14.8. 来自Transformers的双向编码器表示（BERT）
14.9. 用于预训练BERT的数据集
14.10. 预训练BERT


15. 自然语言处理：应用
15.1. 情感分析及数据集
15.2. 情感分析：使用循环神经网络
15.3. 情感分析：使用卷积神经网络
15.4. 自然语言推断与数据集
15.5. 自然语言推断：使用注意力
15.6. 针对序列级和词元级应用微调BERT
15.7. 自然语言推断：微调B

## 远端数据库方式

In [10]:
!docker port milvus-standalone 19530/tcp

0.0.0.0:19530
[::]:19530


In [3]:
from pymilvus import connections, db
 
conn = connections.connect(host="120.79.252.32", port=19530)
database = db.create_database("sample_db")

In [ ]:
测试

In [ ]:
from pymilvus import (
    db,
    MilvusClient,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

#1.创建Milvus客户端
fmt = "\n=== {:30} ===\n"
# 1. connect to Milvus 数据库必须选存在，可通过可视化的管理界面创建数据库
print(fmt.format("1. start connecting to Milvus"))
milvusclient = MilvusClient(uri="http://120.79.252.32:19530", db_name="default")

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530"
)

client.list_databases()


# client.describe_database(
#     db_name="default"
# )



In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

docs[1]


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search)

In [12]:
from langchain_milvus import Milvus
from langchain_community.embeddings import ZhipuAIEmbeddings

embed = ZhipuAIEmbeddings(model="embedding-2",api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 定义连接参数（以本地部署为例）
connection_args = {
    "host": "localhost",   # Milvus服务地址
    "port": "19530"        # 默认端口
}
# 适合生产环境，性能更好
vector_db = Milvus.from_documents(
    documents=docs,
    embedding=embed,
    collection_name="my_collection",  # 自定义集合名
    connection_args=connection_args
)


APIRequestFailedError: Error code: 400, with error text {"error":{"code":"1210","message":"API 调用参数有误，请检查文档。"}}

In [ ]:
query = "What is self-reflection of an AI Agent?"
vectorstore.similarity_search(query, k=1)

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_milvus import Milvus
# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)
print(len(splits))

# 文本嵌入
embed = ZhipuAIEmbeddings(model="embedding-2",api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 测试
# text_1 = "今天天气不错"

# query_result = embeddings.embed_query(text_1)
# print(query_result)

# 路径设置
# persist_directory = './langchain-milvus'

# 向量库创建
vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=embed,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    drop_old=True,  # Drop the old Milvus collection if it exists
)
print(vectordb._collection.count())

# 检索
# question = "图像识别"
# docs = vectordb.similarity_search(question,k=3)
# print(len(docs))
# print(docs[0].page_content)

18


2025-03-31 23:03:54,982 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=65535, message=invalid index type: HNSW, local mode only support FLAT IVF_FLAT AUTOINDEX: )>, <Time:{'RPC start': '2025-03-31 23:03:54.980645', 'RPC error': '2025-03-31 23:03:54.982068'}> (decorators.py:140)
Failed to create an index on collection: LangChainCollection


MilvusException: <MilvusException: (code=65535, message=invalid index type: HNSW, local mode only support FLAT IVF_FLAT AUTOINDEX: )>

18
3
10. 注意力机制
10.1. 注意力提示
10.2. 注意力汇聚：Nadaraya-Watson 核回归
10.3. 注意力评分函数
10.4. Bahdanau 注意力
10.5. 多头注意力
10.6. 自注意力和位置编码
10.7. Transformer


11. 优化算法
11.1. 优化和深度学习
11.2. 凸性
11.3. 梯度下降
11.4. 随机梯度下降
11.5. 小批量随机梯度下降
11.6. 动量法
11.7. AdaGrad算法
11.8. RMSProp算法
11.9. Adadelta
11.10. Adam算法
11.11. 学习率调度器


12. 计算性能
12.1. 编译器和解释器
12.2. 异步计算
12.3. 自动并行
12.4. 硬件
12.5. 多GPU训练
12.6. 多GPU的简洁实现
12.7. 参数服务器


13. 计算机视觉
13.1. 图像增广
13.2. 微调
13.3. 目标检测和边界框
13.4. 锚框
13.5. 多尺度目标检测
13.6. 目标检测数据集
13.7. 单发多框检测（SSD）
13.8. 区域卷积神经网络（R-CNN）系列
13.9. 语义分割和数据集
13.10. 转置卷积
13.11. 全卷积网络
13.12. 风格迁移
13.13. 实战 Kaggle 比赛：图像分类 (CIFAR-10)
13.14. 实战Kaggle比赛：狗的品种识别（ImageNet Dogs）


14. 自然语言处理：预训练
14.1. 词嵌入（word2vec）
14.2. 近似训练
14.3. 用于预训练词嵌入的数据集
14.4. 预训练word2vec
14.5. 全局向量的词嵌入（GloVe）
14.6. 子词嵌入
14.7. 词的相似性和类比任务
14.8. 来自Transformers的双向编码器表示（BERT）
14.9. 用于预训练BERT的数据集
14.10. 预训练BERT


15. 自然语言处理：应用
15.1. 情感分析及数据集
15.2. 情感分析：使用循环神经网络
15.3. 情感分析：使用卷积神经网络
15.4. 自然语言推断与数据集
15.5. 自然语言推断：使用注意力
15.6. 针对序列级和词元级应用微调BERT
15.7. 自然语言推断：微调B